# Devices


Constructing hardware support in Ophyd relies on a pattern of hierarchical composition. Beginning with the base-class, `Device`, functionality may be added via a combination of composition and multiple inheritance.

From a very high level, an Ophyd `Device` is simply a logical container of EPICS process variables. These containers instantiate and attempt connection to their `Component` class attributes, exposing the familiar `get/put` accessors of EPICS.

An example here should help to clarify these notions.

### First, ensure that the EPICS Channel Access context is appropriately initialized

In [ ]:
from ophyd.commands import *

setup_ophyd()

### Import Ophyd `Signal` and `Component` classes

Some imports are required to construct hardware support.

In [ ]:
from ophyd import (EpicsSignal, EpicsSignalRO, PVPositioner, Component as Cpt)

The `CompositeMotor` class below inherits from `PVPositioner`, which is a descendent of the base-class, `Device`. Presumably, it inherits some motor-like behavior as well. The `PVPositioner` expects to have several, self-explanatory attributes: `setpoint`, `readback`, `actuate`, `stop_signal`, and `done`. Here, these attributes are expressed as `EpicsSignals`, as they will connect to the simulated hardware environment.

The utility of this `PVPositioner` is that it affords the capability of constructing a 'motor-like' object, without insisting on a rigid EPICS API (like the Motor Record) to provide the required functionality. The use-cases for the PVPositioner are not limited to positioning systems: these could be used for temperature, pressure, or electrometer controllers too.


`Component`s are factory classes that accept a `Signal`-type class and PV suffix, and mate these with the parent `Device`'s PV prefix. In the example below, `fakemtr` has a PV prefix of 'sim:fake:', which is concatenated with its `setpoint` attribute's suffix of 'mtr-sp' to create a complete PV of 'sim:fake:mtr-sp'. The result is that the following operations are now equivalent:

```python
caput('sim:fake:mtr-sp', 3.14)

fakemtr.setpoint.put(3.14)
```

In [ ]:
class CompositeMotor(PVPositioner):
    setpoint = Cpt(EpicsSignal, 'mtr-sp')
    readback = Cpt(EpicsSignalRO, 'mtr-i')
    actuate = Cpt(EpicsSignal, 'mtr:go-cmd.PROC')
    stop_signal = Cpt(EpicsSignal, 'mtr:stop-cmd.PROC')
    done = Cpt(EpicsSignal, 'mtr:moving-sts')
    
    actuate_value = 1
    stop_value = 1
    done_val = 1

If we instantiate a `CompositeMotor`, we can exercise various methods exposed that make the object resemble a positioner of some type.

In [ ]:
fakemtr = CompositeMotor('sim:fake:', name='fakemtr', egu='mm')

In [ ]:
fakemtr.connected

In [ ]:
fakemtr.position

In [ ]:
fakemtr.move(1.0)

In [ ]:
fakemtr.stop()

In [ ]:
fakemtr.position

The individual class attributes of `fakemtr` are also available for access:

In [ ]:
fakemtr.readback.get()

Examining the repr of `fakemtr.readback`, note that a `Signal`-based `Component` has attributes and methods of its own.


In [ ]:
fakemtr.readback

In fact, some of these attributes are exactly the same as what would be obtained using EPICS calls directly.

In [ ]:
from epics import cainfo

cainfo('sim:fake:mtr-i')